In [7]:
from imports import *
from models import SiameseClassificationNetwork,SiameseNetwork
import data_processing as dp

In [2]:
from transformers import T5Tokenizer, T5Model
import nethook

/home/hrk21/miniconda3/envs/modelediting/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device",device)

device cpu


In [4]:
def get_model(PATH: str,device):

    # Model and tokenizer
    # model_name = "google/t5-small-ssm-nq"
    tokenizer = T5Tokenizer.from_pretrained(PATH, local_files_only=True)
    model = T5Model.from_pretrained(PATH, local_files_only=True)
    model.to(device)
    return model,tokenizer
model_path="/home/hrk21/projects/def-hsajjad/hrk21/datasets/models--google--t5-small-ssm-nq/snapshots/4371c64b6f65176f6663af43066bd094597b1116"
model_t5,tokenizer=get_model(model_path,device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/hrk21/miniconda3/envs/modelediting/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
model=SiameseNetwork(512).to(device)
path_to_folder="/home/hrk21/projects/def-hsajjad/hrk21/datasets/Projector_Networks/Contrastive_Learning_Automated_AVG_EMB/contrastive_mode_500_0/best_model_weights.pth"
state_dict = torch.load(path_to_folder,map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [11]:
def read_dataset_reduced(file_path_read_dataset: str,data_size):
    dataset=[]
    values_list = list(range(1, data_size+1))
    for number in values_list:
        
        # print(json.loads(linecache.getline(file_path_read_dataset, number)))
        # print(linecache.getline(file_path_read_dataset, number).strip())
        data_entry = json.loads(linecache.getline(file_path_read_dataset, number).strip())
        dataset.append(data_entry)
    return dataset
num_samples=10
# file_path_dataset="/home/hrk21/projects/def-hsajjad/hrk21/datasets/counterface_dataset_avg_embedding.jsonl"
# file_path_dataset="/home/hrk21/projects/def-hsajjad/hrk21/datasets/counterface_dataset_special_embedding.jsonl"
file_path_dataset="/home/hrk21/projects/def-hsajjad/hrk21/datasets/counterface_dataset_special_embedding_reduced_500_openai_paraphrases_updated_final_processed.jsonl"
# file_path_datase="/home/hrk21/projects/def-hsajjad/hrk21/datasets/counterface_dataset_avg_embedding.jsonl"
control=0#version of datasplit to be used, 0 based on high sim, 1 on low sim and 3 on random 
label_reversal=False# no longer required cost functions updated
loss="contrastive" # cosine, cosine_crossentropy, contrastive
print("Loading data")
# dataset=read_dataset(file_path_dataset,data_size=num_samples)
dataset=read_dataset_reduced(file_path_dataset,data_size=num_samples) 
print("Loading data completed")
# dp.create_dataset_pairs(
# dataset_paired_train,dataset_paired_test=    create_dataset_pairs_projection(dataset,projector,neightbour_control=0,label_reversal=False)
print("Data Processing")
dataset_paired_train,dataset_paired_test=dp.create_dataset_pairs(dataset,control,label_reversal)#neighbourhood selection type 0 and reverse labels for Constrastive
# dataset_paired_train_openai=dp.data_construct_high_sim(dataset_paired_train,neightbour_control=0,label_reversal=label_reversal,comparison="sim",topk_neg=15,topk_pos=0)
   

Loading data
Loading data completed
Data Processing


In [12]:
l=["encoder.block.4.layer.1.DenseReluDense.wo"]

In [21]:
dataset_paired_train[38][4]

'Google Patents, a product developed by'

In [32]:
sent1="Delta Goodrem, that was formulated in"
# sent2="Delta Goodrem, that originated in"
sent2="Delta Goodrem, a product of"

In [ ]:
{"orignal_prompt": ["Delta Goodrem, that originated in", "Australia"], "edited_prompt": ["Delta Goodrem, that originated in", "India"], "edited_prompt_paraphrases_processed": "Delta Goodrem was from", "edited_prompt_paraphrases_unprocessed": "The first president of the team was doctor Hector Priem. Delta Goodrem was from", "edited_prompt_paraphrases_processed_testing": "Delta Goodrem, that was formulated in", "edited_prompt_paraphrases_unprocessed_testing": "Then, Yuu was supposed to take care of him. Delta Goodrem, that was formulated in", "neighborhood_prompts_high_sim": ["Fortress, that originated in", "K-9 originated in", "Fortress, that was created in", "Sanctum was developed in", "The Libertine, created in"], "neighborhood_prompts_low_sim": ["Don't Say a Word, that was formulated in", "The Pacific was created in the country of", "Strange Bedfellows was created in", "Cut Copy, formulated in", "Until the End of the World, developed in"], "vector_edited_prompt": "", "openai_usable_paraphrases": ["Delta Goodrem, coming from", "Delta Goodrem, with its roots in", "The birthplace of Delta Goodrem was", "The starting point for Delta Goodrem was", "Delta Goodrem, having its origins in", "Delta Goodrem, hailing from", "Delta Goodrem, a product of", "The source of Delta Goodrem was", "Delta Goodrem, which can be traced back to", "Delta Goodrem, that traces its origins back to"]

In [36]:
openai_usable_paraphrases=["Delta Goodrem, coming from", "Delta Goodrem, with its roots in", "The birthplace of Delta Goodrem was", "The starting point for Delta Goodrem was", "Delta Goodrem, having its origins in", "Delta Goodrem, hailing from", "Delta Goodrem, a product of", "The source of Delta Goodrem was", "Delta Goodrem, which can be traced back to", "Delta Goodrem, that traces its origins back to"]

In [37]:
# id=8
# sent2=dataset_paired_train[id][5]
# print(dataset_paired_train[id][5])
# #"Angola is a part of the continent of"#"Johann Wolfgang von Goethe, who has a citizenship from"

In [39]:
#special token
print("Sent1",sent1)
for sent2 in openai_usable_paraphrases:

    with nethook.TraceDict(model_t5, l) as ret:
        inputs=tokenizer(sent1, return_tensors="pt")["input_ids"].to(device)
        outputs = model_t5(inputs, decoder_input_ids=inputs, output_hidden_states=True)
        v1=torch.tensor([[ret[layer_fc1_vals].output[:,-1, :] for layer_fc1_vals in ret][0][0].detach().cpu().numpy().tolist()])

        inputs=tokenizer(sent2, return_tensors="pt")["input_ids"].to(device)
        outputs = model_t5(inputs, decoder_input_ids=inputs, output_hidden_states=True)
        v2=torch.tensor([[ret[layer_fc1_vals].output[:,-1, :] for layer_fc1_vals in ret][0][0].detach().cpu().numpy().tolist()])
        output1, output2 = model(v1,v2)
        print("sent2",sent2)
        print(F.pairwise_distance(output1, output2, keepdim=True)[0][0].item())
        # print(F.pairwise_distance(v1, v2, keepdim=True)[0][0].item())


Sent1 Delta Goodrem, that was formulated in
sent2 Delta Goodrem, coming from
51.218441009521484
sent2 Delta Goodrem, with its roots in
40.691802978515625
sent2 The birthplace of Delta Goodrem was
44.992855072021484
sent2 The starting point for Delta Goodrem was
73.1138687133789
sent2 Delta Goodrem, having its origins in
42.393470764160156
sent2 Delta Goodrem, hailing from
77.70214080810547
sent2 Delta Goodrem, a product of
24.108654022216797
sent2 The source of Delta Goodrem was
37.90758514404297
sent2 Delta Goodrem, which can be traced back to
41.31798553466797
sent2 Delta Goodrem, that traces its origins back to
28.05472755432129


In [34]:
output1, output2 = model(v1,v2)
print(F.pairwise_distance(output1, output2, keepdim=True)[0][0].item())
print(F.pairwise_distance(v1, v2, keepdim=True)[0][0].item())


24.108654022216797
56.38518524169922


In [25]:
# #avg token
# with nethook.TraceDict(model_t5, l) as ret:
#     inputs=tokenizer(sent1, return_tensors="pt")["input_ids"].to(device)
#     outputs = model_t5(inputs, decoder_input_ids=inputs, output_hidden_states=True)
#     v1=torch.tensor([[ret[layer_fc1_vals].output[:,:-1, :].squeeze(0).mean(dim=0).detach().cpu().numpy().tolist() for layer_fc1_vals in ret][0]])

#     inputs=tokenizer(sent2, return_tensors="pt")["input_ids"].to(device)
#     outputs = model_t5(inputs, decoder_input_ids=inputs, output_hidden_states=True)
#     v2=torch.tensor([[ret[layer_fc1_vals].output[:,:-1, :].squeeze(0).mean(dim=0).detach().cpu().numpy().tolist() for layer_fc1_vals in ret][0]])

In [30]:
output1, output2 = model(v1,v2)
print(util.cos_sim(output1, output2))
print(util.cos_sim(v1, v2))

tensor([[0.8994]], grad_fn=<MmBackward0>)
tensor([[0.5724]])


In [37]:
# dataset_paired_test[0][5]

In [27]:
# dataset_paired_test[0][1][:5]